In [5]:
values_table = [
    {'coalition':[0],       'value_total':6.21,      'values':[6.21,     0,          0]},
    {'coalition':[1],       'value_total':4.15,      'values':[0,        4.15,       0]},
    {'coalition':[2],       'value_total':4.15,      'values':[0,        0,          4.15]},
    {'coalition':[0,1],     'value_total':12.08,     'values':[7.07,     5.01,       0]},
    {'coalition':[0,2],     'value_total':12.08,     'values':[7.07,     0,          5.01]},
    {'coalition':[1,2],     'value_total':8.49,      'values':[0,        4.25,       4.25]},
    {'coalition':[0,1,2],   'value_total':16.27,     'values':[7.31,     4.48,       4.48]},
]


class Hedonic:
    def __init__(self,n,values_table):
        self.pi = [[i] for i in range(n)]
        self.values_table = values_table
        self.n = n
    
    def get_current_coalition(self,p):
        for coalition in self.pi:
            if p in coalition:
                return coalition.copy()

    @staticmethod
    def is_equal(coalition1,coalition2):
        return set(coalition1)==set(coalition2)

    def get_player_value(self,p,current_coalition):
        for item in self.values_table:
            if self.is_equal(item['coalition'],current_coalition):
                return item['values'][p]
    
    def update(self,p,base,solution):
        solution = [d for i,d in enumerate(solution) if i!=p]
        for i,coalition in enumerate(self.pi):
            if self.is_equal(base,coalition):
                index_base = i
            if self.is_equal(solution,coalition):
                index_solution = i
        self.pi[index_base].remove(p)
        self.pi[index_solution].append(p)
        self.pi = [sorted(item) for item in self.pi]
        return self.pi

    def converge(self,pi):
        return all([item in self.pi for item in pi])

    def find(self):
        final_pi = self.pi.copy()
        while True:
            for p in range(self.n):
                current_coalition = self.get_current_coalition(p)
                current_value  =  self.get_player_value(p,current_coalition)
                best_option = None
                options = []
                for coalition in self.pi:
                    if not (self.is_equal(coalition, current_coalition) or (p in coalition)):
                        candidate_coalition = coalition + [p]
                        expected_value =  self.get_player_value(p,candidate_coalition)
                        if expected_value > current_value:
                            options.append({"value":expected_value,"coalition":candidate_coalition})
                if options:
                    best_option = sorted(options,key = lambda x: x['value'],reverse=True)[0]
                if best_option:
                    self.pi = self.update(p,current_coalition,best_option["coalition"])
                
            if self.converge(final_pi):
                return self.pi
            else:
                final_pi = self.pi.copy()
        
model = Hedonic(n=3,values_table=values_table)
coalition_formation = model.find()

print(f"Final coalitions = {[coalition for coalition in coalition_formation if coalition][0]}")

Final coalitions = [0, 1, 2]
